## Create a recipe database by scrapping the data from a website

In [15]:
from bs4 import BeautifulSoup #scraping
import requests #for HTTP requests
import re #for regular expression
import time 

import pandas as pd
import numpy as np
import spacy


import random

In [70]:
import unicodedata

### 1. Defining function to scrape the website "allrecipes.com"

In [16]:
def getRecipeLink(url, totalPage):
    recipes_all = []
    for page in range(totalPage):
        if page == 0:
            str_index_page = f"s0t32" + "/Vegetarisch-Rezepte.html"
        else:
            str_index_page = f"s{str(page*30).zfill(3)}t32" + "/Vegetarisch-Rezepte.html"
        
        r = requests.get(base_url + str_index_page) #returns a variable which contains the html doc unstructured
        html = BeautifulSoup(r.content, "html.parser")
        k = 0 #to count recipes per page
        for i in html.findAll('article', {"class": "rsel-item ds-grid-float ds-col-12 ds-col-m-8"}):
            link = i.find("a", attrs={'href': re.compile("https://www.chefkoch.de/rezepte")})
            name = link["data-vars-recipe-title"]
            recipes_all.append({"name" : name, "url" : link['href']})
            k+=1
        print("Extracted {} recipes".format(k))
    return recipes_all

In [17]:
base_url = "https://www.chefkoch.de/rs/"

In [61]:
regex_pattern_numbers_fractions = r"\d+[.,]?[\/]?\d*"
regex_pattern_vulgar_fractions = r"[\u00BC-\u00BE\u2150-\u215E]"

In [21]:
recipes_list_test = getRecipeLink(base_url, totalPage=1)
recipes_list_test

Extracted 30 recipes


[{'name': 'Die perfekten Hamburgerbrötchen',
  'url': 'https://www.chefkoch.de/rezepte/2642351415194117/Die-perfekten-Hamburgerbroetchen.html'},
 {'name': 'Joghurtbombe',
  'url': 'https://www.chefkoch.de/rezepte/2568071401917035/Joghurtbombe.html'},
 {'name': 'Baguette magique',
  'url': 'https://www.chefkoch.de/rezepte/2587261406118751/Baguette-magique.html'},
 {'name': 'Amerikanisches Gurkenrelish für eine Hamburger Sauce',
  'url': 'https://www.chefkoch.de/rezepte/2446211385409096/Amerikanisches-Gurkenrelish-fuer-eine-Hamburger-Sauce.html'},
 {'name': 'Schneemoussetorte mit Rhabarber',
  'url': 'https://www.chefkoch.de/rezepte/1521151257327860/Schneemoussetorte-mit-Rhabarber.html'},
 {'name': 'Grießbrei von Großmutter',
  'url': 'https://www.chefkoch.de/rezepte/914031196710118/Griessbrei-von-Grossmutter.html'},
 {'name': 'Hamburger und Hot Dog Buns',
  'url': 'https://www.chefkoch.de/rezepte/2841831436245733/Hamburger-und-Hot-Dog-Buns.html'},
 {'name': 'KFC Coleslaw',
  'url': 'htt

In [22]:
url_12 = recipes_list_test[12]['url']
r_12 = requests.get(url_12) #returns a variable which contains the html doc unstructured
currRecipe_12 = BeautifulSoup(r_12.content, "html.parser")

In [23]:
url_23 = recipes_list_test[23]['url']
r_23 = requests.get(url_23) #returns a variable which contains the html doc unstructured
currRecipe_23 = BeautifulSoup(r_23.content, "html.parser")

In [68]:
currRecipe_test = BeautifulSoup(requests.get(recipes_list_test[23]['url']).content, "html.parser")

In [37]:
# For vegsoc.org
def get_ingreds_together(currRecipe):
    ingreds_together = []
    uls = currRecipe.find_all("ul", {"class": "ingredients-list__list no-list"})
    for ul in uls:
        lis = ul.find_all("li", {"class": "ingredients-list__item js-ingredient-item"})
        for li in lis:
            text = li.text
            if isinstance(text, str):
                text = text.strip()
                ingreds_together.append(text)
    return ingreds_together

In [26]:
# For chefkoch.de
def get_ingreds_together(currRecipe):
    ingreds_together = []
    tables = currRecipe.find_all("table", {"class": "ingredients table-header"})
    for table in tables:
        lis = ul.find_all("li", {"class": "ingredients-list__item js-ingredient-item"})
        for li in lis:
            text = li.text
            if isinstance(text, str):
                text = text.strip()
                ingreds_together.append(text)
    return ingreds_together

In [43]:
get_ingreds_together(curr____Recipe)

['440ml good quality vegan cider',
 '1 tsp chopped chives',
 '2 tbsp olive oil',
 '2 tbsp white caster sugar',
 '11/2 tbsp English mustard',
 'Salt and pepper, to taste',
 '500g mini baby new potatoes',
 '1 tbsp olive oil',
 '2 whole garlic bulbs, tops cut off to expose the cloves',
 '6 vegan sausages, left whole or cut into halves',
 '2 red onions, cut into wedges',
 '150g Tenderstem broccoli',
 '2 red apples, cut into wedges',
 '100g vegan hollandaise sauce',
 '2 tbsp gherkins, finely chopped',
 '1 tsp chives, chopped',
 '1 tsp chopped flat-leaf parsley',
 'A handful flat-leaf parsley and torn basil']

In [122]:
# for vegsoc.org
def get_ingreds_separately(currRecipe):
    divs = currRecipe.find_all("div", {"class": "ingredients-list__meal-part"})
    if divs is None:
        return "no info"
    try:
        headings = [div.find("h5", {"class": "ingredients-list__meal-part-heading"}).text for div in divs]
    except AttributeError:
        return "no info"
    ingred_lists = [get_ingreds_together(div) for div in divs]
    ingred_lists_dict = dict(zip(headings, ingred_lists))
    return ingred_lists_dict

In [87]:
def separate_quant_units_string(quant_units_string, regex_pattern_numbers_fractions=regex_pattern_numbers_fractions, regex_pattern_vulgar_fractions=regex_pattern_vulgar_fractions, verbose=False):
    number = re.findall(string=quant_units_string, pattern=regex_pattern_numbers_fractions)
    flag_vulgar = False
    if len(number) == 0:
        number = re.findall(string=quant_units_string, pattern=regex_pattern_vulgar_fractions)
        flag_vulgar = True
    number = number[0]
    if flag_vulgar:
        number = str(unicodedata.numeric(number))
    if verbose:
        #print(ingred_quant_text, "\t", number)
        pass
    quant = quant_units_string.replace(number,"").strip()
    return [number, quant]  

In [162]:
# for chefkoch.de
def get_ingreds_separately(currRecipe):
    ingred_lists_dict = {}
    tables = currRecipe.find_all("table", {"class": "ingredients table-header"})
    if tables is None:
        return "no info"
    for table in tables:
        thead = table.find("thead")
        if thead:
            h3_head = thead.find("h3")
            sublist_ingreds_name = h3_head.text
        else:
            sublist_ingreds_name = "just_one_ingred_list"
        ingred_quant_ans_name_tuples_list = []
        tds_left_quant = table.find("tbody").find_all("td",  {"class": "td-left"})
        tds_right_ingred_name = table.find("tbody").find_all("td",  {"class": "td-right"})
        assert len(tds_left_quant) == len(tds_right_ingred_name)
        for td_quant, td_ingred_name in zip(tds_left_quant, tds_right_ingred_name):
            ingred_quant_span = td_quant.find("span")
            if ingred_quant_span is None:
                continue
            
            ingred_ingred_name_span = td_ingred_name.find("span")
            if ingred_ingred_name_span is None:
                continue
            
            ingred_quant_text = td_quant.find("span").text.strip()
            if ingred_quant_text is None:
                ingred_quant_text = td_quant.find("span").find("a").text.strip()
            ingred_name_text = td_ingred_name.find("span").text.strip()
            if ingred_name_text is None:
                ingred_name_text = td_ingred_name.find("span").find("a").text.strip()
            
            # processing quant text 
            flag_vulgar = False
            number = re.findall(string=ingred_quant_text, pattern=regex_pattern_numbers_fractions)
            # if the quantity is either a vulgar fraction or not a number
            if len(number) == 0:
                number = re.findall(string=ingred_quant_text, pattern=regex_pattern_vulgar_fractions)
                flag_vulgar = True
            # if there are no numbers in the quantity column
            if len(number) == 0:
                continue
            number = number[0]
            if flag_vulgar:
                number = str(unicodedata.numeric(number))
            #print(ingred_quant_text, "\t", number)
            quant = ingred_quant_text.replace(number,"").strip()
            ingred_quant_ans_name_tuples_list.append((ingred_quant_text, ingred_name_text, number, quant))
        ingred_lists_dict[sublist_ingreds_name] = ingred_quant_ans_name_tuples_list
    return ingred_lists_dict

In [163]:
ts = currRecipe_12.find_all("table", {"class": "ingredients table-header"})
len(ts)

1

In [164]:
get_ingreds_separately(currRecipe_test_list[6])

{'just_one_ingred_list': [('250                                ml',
   'Vollmilch',
   '250',
   'ml'),
  ('20                                g', 'Frischhefe, 1/2 Würfel', '20', 'g'),
  ('12                                g', 'Zucker', '12', 'g'),
  ('8                                g', 'Salz', '8', 'g'),
  ('25                                g', 'Honig', '25', 'g'),
  ('38                                g', 'Rapsöl', '38', 'g'),
  ('425                                g', 'Weizenmehl Type 550', '425', 'g')],
 'Außerdem: optional': [('1                                m.-großes',
   'Ei(er)',
   '1',
   'm.-großes'),
  ('2                                EL', 'Milch', '2', 'EL')]}

In [97]:
currRecipe_test_list = [BeautifulSoup(requests.get(recipes_list_test[ix]['url']).content, "html.parser") for ix in range(30)]

In [95]:
def get_nutritional_metrics(currRecipe):
    nutritional_metrics_names_list = ["kcal", "protein", "fat", "carbs"]
    divs = currRecipe.find_all("div", {"class": "ds-col-3"})
    nutritional_metrics_keys = []
    nutritional_metrics_values = []
    for div in divs:
        try:
            nutritional_metrics_keys.append(div.find("h5").text)
            nutr_val_list = separate_quant_units_string(div.text.strip().split("\n")[-1].strip())
            nutritional_metrics_values.append(nutr_val_list)
        except AttributeError:
            break
    if divs:
        nutritional_info_dict = dict(zip(nutritional_metrics_keys, nutritional_metrics_values))
    else:
        nutritional_info_dict = dict(zip(nutritional_metrics_names_list, ["no info"]*len(nutritional_metrics_names_list)))
    return nutritional_info_dict

In [98]:
get_ingreds_separately(currRecipe_test_list[23])

1                                Pck. 	 1
1                                Becher 	 1
150                                g 	 150
1 	 1
½                                TL 	 0.5
1                                TL, gestr. 	 1


{'just_one_ingred_list': [('1                                Pck.',
   'Frischkäse',
   '1',
   'Pck.'),
  ('1                                Becher', 'Schmand', '1', 'Becher'),
  ('150                                g', 'Dattel(n), entsteint', '150', 'g'),
  ('1', 'Knoblauchzehe(n)', '1', ''),
  ('½                                TL',
   'Currypulver',
   '0.5',
   '½                                TL'),
  ('1                                TL, gestr.',
   'Chilipaste oder Harissapaste',
   '1',
   'TL, gestr.')]}

In [99]:
def get_keypoints_list(currRecipe):
    a_s = currRecipe.find_all("a", {"class": "ds-tag bi-tags"})
    keypoints_list = [a_item.text.strip() for a_item in a_s]
    return keypoints_list

In [121]:
get_nutritional_metrics(currRecipe_test_list[23])

{'kcal': ['1478', ''],
 'Eiweiß': ['26,05', 'g'],
 'Fett': ['108,93', 'g'],
 'Kohlenhydr.': ['97,32', 'g']}

In [108]:
get_keypoints_list(currRecipe_test_list[0])

['Backen', 'Vegetarisch', 'Brot oder Brötchen', 'Gluten', 'Lactose']

In [58]:
get_keypoints_list(curr____Recipe)

['Dairy-Free', 'Egg-Free', 'Gluten-Free', 'Nut-Free', 'Vegan', 'Wheat-Free']

In [115]:
def get_prep_info_dict(currRecipe):
    prep_info_dict = {}
    small_tag = currRecipe.find("small", {"class": "ds-recipe-meta rds-recipe-meta"})
    spans = small_tag.find_all("span")
    for span in spans:
        raw_text_list = span.text.strip().split()
        info_type = raw_text_list[1]
        mixed_time_info_list = " ".join(raw_text_list[2:]).replace("ca.", "").strip().split()
        time_units = mixed_time_info_list[-1]
        time_quantity = mixed_time_info_list[0]
        prep_info_dict[info_type] = (time_quantity, time_units)
        
    return prep_info_dict

In [116]:
get_prep_info_dict(currRecipe_test_list[0])

{'Arbeitszeit': ('15', 'Minuten'),
 'Ruhezeit': ('2', 'Minuten'),
 'Koch-/Backzeit': ('20', 'Minuten'),
 'Gesamtzeit': ('2', 'Minuten')}

In [117]:
def get_servings_number(currRecipe):
    input_tag = currRecipe.find("input", {"aria-label": "Anzahl der Portionen"})
    return str(input_tag["value"])

In [122]:
get_servings_number(currRecipe_test_list[15])

'1'

In [141]:
def get_instructions(currRecipe):
    divs = currRecipe.find_all("div", {"class": "ds-box"})
    for div in divs:
        children_br = div.findChildren("br", recursive=False)
        children_a = div.findChildren("a", recursive=False)
        if children_br and len(children_a) == 0:
            break
    instructions = div.text.strip().replace("\n", " ").replace("\ue83a", "").replace("\r", "").strip()
    return instructions

In [142]:
get_instructions(currRecipe_test_list[23])

'Die Datteln klein schneiden, den Knoblauch pressen oder nach Wunsch ebenfalls klein schneiden.  Den Frischkäse mit dem Schmand glatt rühren, die Datteln und den Knoblauch hineingeben. Anschließend noch die Chilipaste und das Currypulver dazugeben.  Hinweise: Wer Chilipaste nicht mag, kann Paprikapulver nehmen.'

In [168]:
def getEachRecipe(recipesList):
    for ix in range(len(recipesList)):
        url = recipesList[ix]['url']
        r = requests.get(url) #returns a variable which contains the html doc unstructured
        currRecipe = BeautifulSoup(r.content, "html.parser")
        #there are two versions of the website, the old and new have different html structure
        #they require different parsing
        ver = 0
        recipe_name = recipesList[ix]["name"]
        recipesList[ix]['id'] = str(ix+1).zfill(4)
        recipesList[ix]['name'] = recipe_name
        
        instructions = get_instructions(currRecipe)
        recipesList[ix]['instructions'] = instructions

        ingred_separate_lists = get_ingreds_separately(currRecipe)
        recipesList[ix]['ingred_separate_lists'] = ingred_separate_lists
        
        key_points_list = get_keypoints_list(currRecipe)
        recipesList[ix]['key_points_list'] = key_points_list
               
        nutritional_info = get_nutritional_metrics(currRecipe)
        recipesList[ix]['nutritional_info_dict'] = nutritional_info       

        
        preparation_info_dict = get_prep_info_dict(currRecipe)
        recipesList[ix]['preparation_info_dict'] = preparation_info_dict
        
        recipesList[ix]['servings_number'] = get_servings_number(currRecipe)
        
        print("\r", "Adding {0} recipe".format(ix + 1), end=" ")
    return recipesList

In [149]:
recipes_test = getRecipeLink(base_url, 1)

Extracted 30 recipes


In [165]:
data_test = getEachRecipe(recipes_test)

 Adding 30 recipe 

In [166]:
import json

In [169]:
with open('chefkoch_de_veg_test.json', 'w', encoding="utf-8") as fp:
    json.dump(data_test, fp, indent=4, ensure_ascii=False)

In [170]:
recipes_prod = getRecipeLink(base_url, 33)

Extracted 30 recipes
Extracted 30 recipes
Extracted 30 recipes
Extracted 30 recipes
Extracted 30 recipes
Extracted 30 recipes
Extracted 30 recipes
Extracted 30 recipes
Extracted 30 recipes
Extracted 30 recipes
Extracted 30 recipes
Extracted 30 recipes
Extracted 30 recipes
Extracted 30 recipes
Extracted 30 recipes
Extracted 30 recipes
Extracted 30 recipes
Extracted 30 recipes
Extracted 30 recipes
Extracted 30 recipes
Extracted 30 recipes
Extracted 30 recipes
Extracted 30 recipes
Extracted 30 recipes
Extracted 30 recipes
Extracted 30 recipes
Extracted 30 recipes
Extracted 30 recipes
Extracted 30 recipes
Extracted 30 recipes
Extracted 30 recipes
Extracted 30 recipes
Extracted 30 recipes


In [171]:
import time

In [172]:
start_scrapping = time.time()
data_prod = getEachRecipe(recipes_prod)
end_scrapping = time.time()

duration_scrapping_minutes = (end_scrapping - start_scrapping) / 60
print("Scrapping took {} minutes".format(duration_scrapping_minutes))

 Adding 990 recipe Scrapping took 9.146813412507376 minutes


In [173]:
with open('chefkoch_de_prod.json', 'w', encoding="utf-8") as fp:
    json.dump(data_prod, fp, indent=4, ensure_ascii=False)